# Storing Information in Databases

### Why use Databases?

A) Databases provide a special way to store data that involves saving granular pieces of information that can be recalled later. <br /><br />
B) Databases allow you to manipulate data without loading the entire dataset into memory. While JSON, spreadsheet etc have this limitation. <br /><br />
C) Database makes it easier to reuse data. <br /><br />
D) In general, database complies with ACID (Atomicity, Consistency, Isolation, Durability) <br />


### Talking to Databases with SQL

SQL (Structured Query Language) is the language used to talk to databases. <br />
<b>DONT PANIC ABOUT HAVING TO LEARN YET ANOTHER PROGRAMMING LANGUAGE</b><br />
We will be using a limited subset of what SQL has to offer. So this wont be like learning a new language.

SQL is written in statements. These statements tell the database what you want to do and what data you want to work with. The database then takes the data, does what you want, and either saves your requested change or gives you the data.

In relational databases, data are stored in different tables or tuples. You can imagine a table like an excel spread sheet. There are attributes or fields that define column names and data types and the rows are the actual data.

Each database vendor offer its own version of SQL that contains subtle differences. Some have extra functionality. Some have more tools to deal with dates or time. A few have slightly different format. Once you learn one it is fairly easy to read another database's SQL statements.

There are many kinds of databases. Some are free, and some are so expensive that they are rarely seen outside of large companies. Some require a complex setup, wherras others are extremely easy to install. For our purposes, we will be using a lightweight and free database called SQLite(pronounced S-Q-L lite)

### SQLite on a Mac

If you using a Mac, you should already have SQLite on your system. Open a terminal and type sqlite3. You should see something close to the following:<br /><br />
SQLite version 3.8.11 2015-07-27 13:49:41<br />
Enter ".help" for usage hints.<br />
Connected to a transient in-memory database.<br />
Use ".open FILENAME" to reopen on a persistent database.<br />
sqlite><br />

Type '.exit' to get out of this dialog.

### Installing SQLite on Windows

SQLite does not come installed on Windows, so you will have to install it yourself. <br />
First go to SQLite website download page (http://www.sqlite.org/download.html) <br />
There, you will find many links to files for a variety of operating systems. You are going to need the one under precompiled binaries for Windows. This is the one for a command-line shell, not the one that contains DLLs or is an analyzer. <br /><br />
Create a folder named <b>sqllite</b>(or whatever you like) and unzip the file you downloaded there. You should see <b>sqlite3.exe</b> there. <br />
Create a folder named <b>sqllite_databases</b>(or whatever you like) and this folder is the folder for having all your databases.<br /><br />
Open a command-line and run the sqlite3.exe and that gives you console for sqlite. (You can put the path in PATH envromental variable so you can run it from anywhere in command-line)<br />
Few command to start with:<br />
.exit (exit the shell or console)<br />
.open (creates a database or opens an existing one)<br />
i.e.<br />
<b>.open testdb.sqlite </b> (creates a database in the directory where you running this command)<br />
<b>.open ..\sqlite-databases\testdb.sqlite </b>(creates the database in folder we created for our databases)<br />
<b>.databases </b>(shows your database info)<br />
<b>.tables </b>(shows you all yables in the database)<br />
<b>.help </b>(gives you help on different commands)<br />





### SQLite GUI Client

One of the clients you can use is FireFox add-ons SQLite manager. (I am sure if you Google, you can find tons of other ones)<br />
Goto your FireFox add-ons and search for SQLite manager and install it. <br />
Open your FireFox and click F10 and you should see Tools tab and clicking on that you should see SQL manager.<br />
You can use the SQLite manager to connect to database, create tables, run queries etc.

### SQLite Is Serverless

Most SQL database engines are implemented as a separate server process. Programs that want to access the database communicate with the server using some kind of interprocess communication (typically TCP/IP) to send requests to the server and to receive back results. SQLite does not work this way. With SQLite, the process that wants to access the database reads and writes directly from the database files on disk. There is no intermediary server process. 

### Making a Table

As mentioned earlier, the data you put in a database is always held in a table. A database can have one table or a hundred. You cannot just throw any data into a database. You have to tell the database what sort of data it can expect.

#### SQLite Data Types

<b>Null</b>     (An empty value like Python None, example: NULL)<br />
<b>Integer</b>  (A whole number, just like Python, example: -20)<br />
<b>Real</b>     (A float, example: 1.0)<br />
<b>Text</b>     (A string, example "Hello")<br />
<b>Blob</b>     (binary large object, example: An image, zip file)<br />

https://www.sqlite.org/datatype3.html

Now Lets create a table using Python and SQLite:

In [ ]:
import sqlite3
# create a connection to an existing database. It creates a new one if it did not exists.
# extension of database can be anything.
# it creates database in same folder you are running iPython notebook file. You can use absolute path if you want.
conn = sqlite3.connect('mytestdb.sqlite')
# get the cursor. A cursor is what we use to move around database, execute sql and get data. 
cursor = conn.cursor()
# write your SQLite query 
sql = 'create table students(name text, username text, id int)'
cursor.execute(sql)
cursor.close()

Goto to folder you are running this iPython notebook file and check if database created.

Use FireFox or any other client to see if you can connect to database and see the tables

### Adding Data

In [ ]:
import sqlite3

conn = sqlite3.connect('mytestdb.sqlite')
cursor = conn.cursor()
print "Let's input some students!"
while True:
    # getting user input as string, this is Python built in function
    name = raw_input('Student\'s name:')
    username = raw_input('Student\'s username:')
    id_num = raw_input('Student\'s id number:')
    sql = 'insert into students (name, username, id) values(:st_name, :st_username, :id_num)'
    cursor.execute(sql, {'st_name':name, 'st_username':username, 'id_num':id_num})
    # you need to commit your transaction otherwise it get lost. It is cached in memory
    conn.commit()
    cont = raw_input("Another student? ")
    if cont[0].lower() == 'n':
        break
cursor.close()


### Querying the Database

In [ ]:
import sqlite3

conn = sqlite3.connect('mytestdb.sqlite')
cursor = conn.cursor()
sql = "select * from students"
results = cursor.execute(sql)
# returns a list
all_students = results.fetchall()
for student in all_students:
    print(student)

In [ ]:
# if you only want the names then simply change the query to extract the names only
import sqlite3

conn = sqlite3.connect('mytestdb.sqlite')
cursor = conn.cursor()
sql = "select name from students"
results = cursor.execute(sql)
all_students = results.fetchall()
for student in all_students:
    print(student)

### A real World Example
   

We want to enter our inventory into database, so following is the complete program:

In [ ]:
import sqlite3

def open_database(db_name):
    conn = sqlite3.connect(db_name)
    return conn

def create_table(cursor):
    sql = 'create table ingrediants(title text, amount float, description text)'
    try:
        cursor.execute(sql)
    except:
        pass # pass is a statement that does not do anything (keyword)
    
def add_ingrediants(cursor):
    while True:
        ingrediants = raw_input("Name of ingrediants (q to quit): ")
        if ingrediants.lower() != 'q':
            num = raw_input("Number in storage: ")
            description = raw_input("description: ")
            sql = 'insert into ingrediants \
                   (title, amount, description) \
                    values \
                    (:title, :amount, :description)'
            cursor.execute(sql, {"title":ingrediants, "amount":num, "description":description})
            print "Added!"
        else:
            print "Okay, quitting"
            break
            
def main():
    conn = open_database('inventory.db')
    cursor = conn.cursor()
    create_table(cursor)
    add_ingrediants(cursor)
    conn.commit()
    conn.close()

# __name__ is a variable that will be set to __main__ if you run this module
# if you import this module the __name__ will be set to name of that module while 
# executing
# this check for making sure you can execute this code only if you execute directly, if you import this module
# then main() will not trigger as the if condition will be False
if __name__ == "__main__":
    main()
        

### Using SQL to Get More out of Databases

Let say we have a database holding video games in a table called games.

In [ ]:
import sqlite3
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'create table games(title text, rating text, system text, year int)'
cursor.execute(sql)
cursor.close()

In [ ]:
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'insert into games (title, rating, system, year) values(:title, :rating, :system, :year)'
cursor.execute(sql, {'title':'Forza Horizon','rating':'T', 'system':'3DS', 'year':2011})
cursor.execute(sql, {'title':'Tales of the Abyss','rating':'E10+', 'system':'3DS', 'year':2012})
cursor.execute(sql, {'title':'Adventure Time','rating':'T', 'system':'3DS', 'year':2011})
cursor.execute(sql, {'title':'Hollywood Crimes','rating':'E', 'system':'360', 'year':2011})
cursor.execute(sql, {'title':'Forza Motorsport 4','rating':'E', 'system':'360', 'year':2012})
cursor.execute(sql, {'title':'Sonic Generations','rating':'T', 'system':'360', 'year':2012})
cursor.execute(sql, {'title':'ZhuZhu Pets','rating':'E', 'system':'wii', 'year':2012})
conn.commit()
cursor.close()


#### Checking for Equality

In [ ]:
import sqlite3
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'select * from games where rating = "E"'
results = cursor.execute(sql)
e_games = results.fetchall()
e_games


#### Check for Inequality

In [ ]:
import sqlite3
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'select * from games where rating != "E"'
results = cursor.execute(sql)
not_e_games = results.fetchall()
not_e_games

#### Using LIKE to Find Similar Items

In [ ]:
import sqlite3
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'select * from games where rating like "E%"'
results = cursor.execute(sql)
all_e_games = results.fetchall()
all_e_games

#### Using NOT LIKE to Find Nonsimilar Items

In [ ]:
import sqlite3
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'select * from games where rating not like "E%"'
results = cursor.execute(sql)
non_e_games = results.fetchall()
non_e_games

#### Querying with Greater Than and Less Than

In [ ]:
import sqlite3
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'select * from games where year > 2011'
results = cursor.execute(sql)
games = results.fetchall()
games

In [ ]:
import sqlite3
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'select * from games where year <= 2011'
results = cursor.execute(sql)
games = results.fetchall()
games

#### Sorting with ORDER BY

In [ ]:
import sqlite3
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'select title from games order by title desc'
results = cursor.execute(sql)
games = results.fetchall()
games

#### Getting Unique Items with DISTINCT

In [ ]:
import sqlite3
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'select distinct system from games'
results = cursor.execute(sql)
systems = results.fetchall()
systems

#### Updating Records with UPDATE

In [ ]:
import sqlite3
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'update games set title = "Adventure Time, Dark Moon" where title = \
                                 "Adventure Time"'
cursor.execute(sql)
conn.commit()
sql = 'select * from games where title like "Ad%"'
results = cursor.execute(sql)
print results.fetchall()


#### Deleting Records with DELETE

In [ ]:
import sqlite3
conn = sqlite3.connect('gamesdb.sqlite')
cursor = conn.cursor()
sql = 'delete from games where title = "Sonic Generations"'
cursor.execute(sql)
conn.commit()
sql = 'select * from games'
results = cursor.execute(sql)
print results.fetchall()

### Does Python work with other databases?
It does but you have to install a module or library to work with it. A popular toolkit is SQLAlchemy, which works with many popular databases, such as MySQL and Oracle. You can find SQLAlchemy at http://www.sqlalchemy.org <br />

If SQLAlchemy does not cover your database, the next best option is to go to PyPi (https://pypi.python.org/pypi) and search for your particular database.

### In Class Lab Assignment

Based on Inventory example, write a Python code that presents the user with following funcational menu:

What do you want to do?<br />
A- Add an ingredient<br />
S- Search for an ingredient<br />
L- List all ingredients<br />
Q- Quite<br />